# Python Geospatial Fundamental: Geoprocessing and Analysis

* * * 

<div class="alert alert-success">  
    
### Learning Objectives 
    
* Apply more advanced Python libraries for interactive visualization.
* Choose domain-specific spatial datasets to create your own maps.
</div>

### Sections
1. [Mapping and Spatial Analysis (cont'd)](#section1)
2. [Interactive Visualization](#section2)
3. [API, Vector and Raster Data Sources](#section3)
4. [Create Your Own Map](#section4)

<a id='section1'></a>

# 1. Mapping and Spatial Analysis (cont'd)

## Spatial Join and Aggregation

A **spatial join**, is the process by which we can leverage the spatial relationships between distinct datasets to merge their information into a new, synthetic dataset. This operation can be thought as the spatial equivalent of an **attribute join**, in which multiple tabular datasets can be merged by aligning matching values in a common column that they both contain. If you've done data wrangling in Python with `pandas`, you've probably performed an attribute join at some point.

We'll start by developing an understanding of this operation using census tract data.

In [1]:
import pandas as pd
import geopandas as gpd

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline  

In [6]:
# import school data
alameda_school = gpd.read_file('../data/alameda_school.shp')

# import census tract data
tracts_gdf = gpd.read_file("zip://../data/Census/Tracts/cb_2013_06_tract_500k.zip")

<a id='section2'></a>

# 2. Interactive Visualization

Leaflet map

<a id='section3'></a>

# 3.🎬 **Demo**: API, Vector and Raster Data Sources

<a id='section4'></a>

# 4. 🥊 Challenge: Create Your Own Map

In [ ]:
# You may have some starter code for the challenge that you can put in its own cell.
# Always have a following cell that says "YOUR CODE HERE" with a few empty lines beneath it, 
# so that attendees know where to put their code.

In [ ]:
# YOUR CODE HERE




<details><summary><a>Expandable sections</a></summary>
Expandable sections like these can be used for hints, or for information that is optional or more advanced, and you do not want to immediately expose to all workshop participants. Use these expandable sections scarcely!
</details>


<div class="alert alert-success">

## ❗ Key Points

* EDIT ME. Recap the most useful information in this notebook. Keep it short and use action words. Min 3, max 8.
* EDIT ME. Recap the most useful information in this notebook. Keep it short and use action words. Min 3, max 8.
* EDIT ME. Recap the most useful information in this notebook. Keep it short and use action words. Min 3, max 8.

</div>